In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import sys
from sklearn.cross_validation import cross_val_predict

from sklearn.pipeline import Pipeline

# basic natural language feature extraction
import sklearn.feature_extraction.text as fet

# used for train/test splits and cross validation
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV

# used to impute mean for data and standardize for computational stability
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

# logistic regression is our favourite model ever
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LassoCV

# used to calculate AUROC/accuracy
from sklearn import metrics

# used to create confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import cross_val_score

# gradient boosting - must download package https://github.com/dmlc/xgboost
#import xgboost as xgb

# default colours for prettier plots

col = [[0.9047, 0.1918, 0.1988],
    [0.2941, 0.5447, 0.7494],
    [0.3718, 0.7176, 0.3612],
    [1.0000, 0.5482, 0.1000],
    [0.4550, 0.4946, 0.4722],
    [0.6859, 0.4035, 0.2412],
    [0.9718, 0.5553, 0.7741],
    [0.5313, 0.3359, 0.6523]];
marker = ['v','o','d','^','s','o','+']
ls = ['-','-','-','-','-','s','--','--']
%matplotlib inline

from __future__ import print_function

In [ ]:
# read data from the got-data.ipynb file
# most of it is scrapped from the game of thrones tv show wiki
df = pd.read_csv('data/got_data_valid_characters.csv', sep=',', index_col=0)

# print out the data we have
print('Total of {} characters.'.format(df.shape[0]))
df.head(n=3)

```python
# define a matrix of relationships ... realized this is harder than it looks!
# 0 - not related
# 1 - father
# 2 - mother
# 3 - sibling
# 4 - child
# 5 - other
fam = pd.DataFrame(np.zeros([df.shape[0],df.shape[0]]), index=df.index, columns=df.index)
for i, char in enumerate(fam.index):
    curr_fam = df.loc[char, 'Family']
    if curr_fam is np.nan or curr_fam is None:
        continue
    
    # otherwise, let's parse their family tree
    common_fam = [filter(lambda x: x in curr_fam, sublist) for sublist in fam.index]
    print(curr_fam)
    #print(common_fam)
    break
```

In [ ]:
# define some useful subfunctions

# given a dataframe/regex phrase, this counts the number of times the regex appears
def count_words(s, phrase):
    if s is np.nan:
        return 0
    else:
        return len(re.findall(phrase,s, re.IGNORECASE))

# this calls the above function and adds the data to the given dataframe
def add_data(df_data, column_name, phrase, txt_col=None):
    if txt_col is None:
        txt_col = ['Background','Season 1','Season 2', 'Season 3', 'Season 4']
        
    df_new = df[txt_col].applymap(lambda x: count_words(x,phrase))
    df_data[column_name] = df_new[txt_col].sum(axis=1)
    return df_data

def add_allegiance(df, df_has_allegiance):
    
    houses = ['House Lannister', 'House Stark', 'House Targaryen', 'House Frey', 
              'House Greyjoy', "Night's Watch", 'House Martell', 'Free Folk', 'House Forrester',
              'House Bolton', 'House Baratheon', 'Kingsguard']
    
    for h in houses:
        tmp = np.zeros(df.shape[0])
        val = df_has_allegiance['Allegiance'].values
        for i in range(df.shape[0]):
            if val[i] is np.nan:
                continue
            elif h in val[i]:
                tmp[i] = 1.0
        df[h] = tmp
        
    return df

In [ ]:
def extract_data(df, s=3):
    # given the dataframe and the season, extract some features
    # only look at background // season 1
    
    # by default, extract data up to and including season 3, i.e. s=3
    txt_col = ['Background','Season 1','Season 2','Season 3','Season 4',' Season 5']
    
    txt_col = txt_col[0:(s+1)]
    
    # initialize the dataframe
    idxData = np.ones(df.shape[0], dtype=bool)
    df_data = df.loc[idxData, ['Status','Season(s)']]

    # start adding data based on counting the frequency of words

    # count the number of hyphens as a surrogate as the number of in-show family member
    # in the data, the family members appear as "sister - Sansa Stark, father - Eddard Stark," .. etc
    df_data = add_data(df_data, 'family_members',' - ', txt_col = ['Family'])

    #df_data = add_allegiance(df_data, df)
    
    # number of seasons the character appeared in
    df_data = add_data(df_data, 'number_of_seasons','[1-' + str(s) + ']', txt_col = ['Season(s)'])

    # various word "types" across all the data
    df_data = add_data(df_data, 'violent_words','(knight|warrior|sword|axe|spear|kill|murder|fight|assassinate)')
    df_data = add_data(df_data, 'sexy_words','(sex|naked|love|slept|nude|kiss)')
    df_data = add_data(df_data, 'fun_words','(beer|wine|glass|drunk|inebriate)')
    df_data = add_data(df_data, 'number_of_words','\w+')

    # remove people who died before current season
    idxRemove = np.zeros(df_data.shape[0],dtype=bool)
    for i, val in enumerate(df_data['Status'].values):
        char = df_data.index[i]
        if val is None or val is np.nan:
            continue
        if ('Dead' in val) | ('Deceased' in val):
            if len(re.findall('[' + str(s+1) + '-5]', df_data.loc[char, 'Season(s)'])) > 0:
                continue
            else:
                idxRemove[i] = True

    df_data = df_data.loc[~idxRemove]

    df_data.drop('Season(s)',axis=1,inplace=True)
    return df_data
    

In [ ]:
df_data = extract_data(df, s=4)
df_data.head(n=2)

In [ ]:
# prep data
X = df_data.values[:,1:df_data.shape[1]]

# alive until proven dead !
y = np.zeros(df_data.shape[0])
for i, val in enumerate(df_data['Status'].values):
    if val is None or val is np.nan:
        continue
    if 'Dead' in val:
        y[i] = 1.0
    elif 'Deceased' in val:
        y[i] = 1.0
        
# workaround cross_val_predict returning 0s/1s - calling this makes it return probabilities
class proba_logreg(LogisticRegression):
    def predict(self, X):
        return LogisticRegression.predict_proba(self, X)

# cross-validation performance
mdl = "logreg"
model = LogisticRegression(fit_intercept=True)
estimator = Pipeline([("imputer", Imputer(missing_values='NaN',
                                      strategy="mean",
                                      axis=0)),
                  ("scaler", StandardScaler()),
                  ("lr", model)])
scores = cross_val_score(estimator, X, y, scoring='roc_auc',cv=5)
print('{:10s} {:5g} [{:5g}, {:5g}]'.format("lr", np.mean(scores), np.min(scores), np.max(scores) ))

# plot a roc curve by getting cross-validation predictions
predicted = cross_val_predict(proba_logreg(), X, y, cv=10)
predicted = predicted[:,1]

plt.figure(figsize=[9,9])
ax = plt.gca()
fpr, tpr, thresholds = metrics.roc_curve(y, predicted, pos_label=1)
plt.plot(fpr, tpr, 'bo-',lw=2,markersize=12,
        label=mdl + ' ' + '%0.3f' % metrics.auc(fpr, tpr))
plt.xlabel('False positive rate',fontsize=14)
plt.ylabel('True positive rate',fontsize=14)
plt.legend(loc='lower right')
plt.title('Mortality prediction',fontsize=16)
plt.show()

In [ ]:
sort_indices = np.argsort(predicted)

for i in range(20):
    idx = sort_indices[-i-1]
    print('{:20s}: {:0.5f} (outcome: {})'.format(df_data.index[idx],
                                        predicted[idx],
                                       y[idx]))

# TF-IDF

Let's try TF-IDF of the same data !

In [ ]:
def extract_data_text(df, s=3):
    # given the dataframe and the season, extract some features
    # only look at background // season 1
    
    # by default, extract data up to and including season 3, i.e. s=3
    txt_col = ['Background','Season 1','Season 2','Season 3','Season 4',' Season 5']
    
    txt_col = txt_col[0:(s+1)]
    
    # initialize the dataframe
    idxData = np.ones(df.shape[0], dtype=bool)
    df_data = df.loc[idxData, ['Status','Season(s)']]

    # === ADD DATA === #

    tmp = None
    for txt in txt_col:
        if tmp is None:
            tmp = df[txt]
        else:
            tmp = tmp.str.cat(df[txt].fillna('') , sep='\n')

    df_data['text'] = tmp.fillna('')
    
    # remove people who died before current season
    idxRemove = np.zeros(df_data.shape[0],dtype=bool)
    for i, val in enumerate(df_data['Status'].values):
        char = df_data.index[i]
        if val is None or val is np.nan:
            continue
        if ('Dead' in val) | ('Deceased' in val):
            if len(re.findall('[' + str(s+1) + '-5]', df_data.loc[char, 'Season(s)'])) > 0:
                continue
            else:
                idxRemove[i] = True

    df_data = df_data.loc[~idxRemove]
    
    # now we have the data frame ... get X and y matrices
    X = df_data.values[:,1:df_data.shape[1]]

    # alive until proven dead !
    y = np.zeros(df_data.shape[0])
    for i, val in enumerate(df_data['Status'].values):
        if val is None or val is np.nan:
            continue
        # check if they have any seasons after the next one
        # this is to confirm that, if they died, they died in the next season
        if len(re.findall('[' + str(s+2) + '-' + str(np.max([s+2,5])) + ']',
                          df_data.iloc[i]['Season(s)'])) == 0:
            if 'Dead' in val:
                y[i] = 1.0
            elif 'Deceased' in val:
                y[i] = 1.0
        
    #print('Died in season {}'.format(s+1))
    #for i,val in enumerate(y):
    #    if val==1.0:
    #        print('{}'.format(df_data.index[i]))
    return X, y

In [ ]:
X, y = extract_data_text(df, s=3)

In [ ]:
tf = fet.TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0, stop_words = 'english')
tfidf_matrix =  tf.fit_transform(df_data['text'])
feature_names = tf.get_feature_names()
print(len(feature_names))
print('Example phrases:')
for i in range(5):
    print(feature_names[i+50])
    
print(tfidf_matrix.shape)

In [ ]:
tf = fet.TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0, stop_words = 'english')
mdl = LassoCV(cv=5,fit_intercept=True)


pipeline = Pipeline([
    ('tfidf', tf),
    ('clf', mdl),
])

In [ ]:
###############################################################################
# define a pipeline combining a text feature extractor with a simple
# classifier
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}